In [ ]:
import pandas as pd
import numpy as np


In [ ]:
credits=pd.read_csv("credits.csv",error_bad_lines=False,engine="python")

<ipython-input-149-876017047762>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  credits=pd.read_csv("credits.csv",error_bad_lines=False,engine="python")


In [ ]:
movies=pd.read_csv("movies_metadata.csv",error_bad_lines=False,engine="python")

<ipython-input-150-e7f0b74bc5e4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  movies=pd.read_csv("movies_metadata.csv",error_bad_lines=False,engine="python")


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [ ]:
merged=pd.merge(movies,credits,on="id")

ValueError: ignored

In [ ]:
merged.columns

In [ ]:
movies_cleaned=merged.drop(columns=["homepage","production_countries"],axis=1)

In [ ]:
movies_cleaned['overview']=movies_cleaned['overview'].fillna("")

# Content based recommendation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfv=TfidfVectorizer(encoding="unicode",min_df=3,max_features=None,
                    strip_accents="unicode",analyzer="word",token_pattern=r'\w{1,}',
                    ngram_range=(1,3),stop_words="english")

In [ ]:
tfv_matrix= tfv.fit_transform (movies_cleaned['overview'])

In [ ]:
tfv_matrix

<1028x2641 sparse matrix of type '<class 'numpy.float64'>'
	with 20932 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [ ]:
sig[0]

array([0.76175313, 0.76160152, 0.7615961 , ..., 0.76159416, 0.76159416,
       0.76159588])

In [ ]:
#Reverse mapping of idices and titles
indices=pd.Series(movies_cleaned.index,index=movies_cleaned['original_title']).drop_duplicates()

In [ ]:
indices

original_title
Toy Story                         0
Jumanji                           1
Grumpier Old Men                  2
Waiting to Exhale                 3
Father of the Bride Part II       4
                               ... 
The Proprietor                 1023
Normal Life                    1024
Get on the Bus                 1025
Shadow Conspiracy              1026
Jude                           1027
Length: 1028, dtype: int64

In [ ]:
indices['Before Sunrise']

212

In [ ]:
sig[212]

array([0.76159416, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [ ]:
list(enumerate(sig[indices['Before Sunrise']]))

[(0, 0.7615941559557649),
 (1, 0.7615941559557649),
 (2, 0.7615941559557649),
 (3, 0.7615941559557649),
 (4, 0.7615941559557649),
 (5, 0.7615941559557649),
 (6, 0.7615941559557649),
 (7, 0.7616035947956975),
 (8, 0.7615941559557649),
 (9, 0.7615941559557649),
 (10, 0.7615941559557649),
 (11, 0.7615941559557649),
 (12, 0.7615941559557649),
 (13, 0.7616014420091739),
 (14, 0.7615941559557649),
 (15, 0.7615941559557649),
 (16, 0.7615941559557649),
 (17, 0.7616063846376004),
 (18, 0.7615941559557649),
 (19, 0.7615941559557649),
 (20, 0.7615975893292646),
 (21, 0.7615941559557649),
 (22, 0.7615941559557649),
 (23, 0.7615941559557649),
 (24, 0.7615941559557649),
 (25, 0.7616022598356614),
 (26, 0.7615941559557649),
 (27, 0.7615999999117843),
 (28, 0.7615941559557649),
 (29, 0.7616036104225359),
 (30, 0.7615994793723785),
 (31, 0.7615941559557649),
 (32, 0.7615941559557649),
 (33, 0.7615941559557649),
 (34, 0.7615982851112807),
 (35, 0.761598390980341),
 (36, 0.7616028086509045),
 (37, 0.7615

In [ ]:
indexes=sorted(list(enumerate(sig[indices['Before Sunrise']])),key=lambda x:x[1],reverse=True)[1:6]
index_list=[i[0] for i in indexes]

In [ ]:
movies_cleaned['original_title'].iloc[index_list]

599                    Century
859                      Bogus
671            L'oeil de Vichy
566            Foreign Student
334    While You Were Sleeping
Name: original_title, dtype: object

In [ ]:
def giv_rec(title,sig=sig):
  sig_scores=sorted(list(enumerate(sig[indices[title]])),key=lambda x:x[1],reverse=True)
  selected_sigscores=sig_scores[1:6]
  movies_indices=[i[0] for i in selected_sigscores]
  return movies_cleaned['original_title'].iloc[movies_indices]


In [ ]:
giv_rec("Home Alone")

531    A Simple Twist of Fate
213             Billy Madison
371            The River Wild
613        Ma saison préférée
748                      Jack
Name: original_title, dtype: object

In [ ]:
# Note : Same kind of recommendation can be achived with cosine similarity only difference it makes is that similar movies are movies with the lowest distances
